### Model Parameter 설정

In [2]:
%pip install -q langchain langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [1]:
from dotenv import load_dotenv
# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

True

### 1) 모델 클래스 유형

#### LLM

In [4]:
from langchain_openai import OpenAI

# model을 지정하지 않아도 실행되는 이유는 LangChain이 내부적으로 기본값을 설정함
llm = OpenAI()
print(llm.model_name)  # 기본 모델 확인

result = llm.invoke("한국의 대표적인 관광지 3군데를 추천해 주세요.")
print(type(result))
print(result)

gpt-3.5-turbo-instruct
<class 'str'>


1. 경복궁
경복궁은 조선 왕조의 주요 궁궐로, 서울에 위치하고 있습니다. 국내외 관광객들에게 인기 있는 곳으로, 조선 왕실의 역사와 문화를 살펴볼 수 있습니다. 특히 봄과 가을에는 아름다운 조경과 함께 청와대 앞 국민청원 체험, 전통문화 체험 등 다양한 이벤트가 열립니다.

2. 제주도
제주도는 우리나라의 남쪽에 위치한 섬으로, 자연이 아름다운 곳으로 유명합니다. 섬 전체가 유네스코 세계자연유산으로 지정되어 있으며, 화산섬이기 때문에 다양한 화산


#### ChatModel

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

#chat = ChatOpenAI(model="gpt-3.5-turbo-0125")
chat = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.7
)

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 여행 전문가입니다."),
    ("human", "{user_input}"),
])

chain = chat_prompt | chat
result = chain.invoke({"user_input": "안녕하세요? 한국의 대표적인 관광지 3군데를 추천해 주세요."})

print(type(result))
print(result.content)

<class 'langchain_core.messages.ai.AIMessage'>
한국의 대표적인 관광지로는 첫 번째로 **경복궁**이 있습니다. 경복궁은 조선 시대의 궁궐로, 서울에 위치해 있습니다. 이곳은 한국의 역사와 문화를 경험할 수 있는 장소로, 많은 관광객이 방문합니다.

두 번째로는 **제주도**가 있습니다. 제주도는 한국의 남쪽에 위치한 섬으로, 아름다운 자연 경관과 독특한 문화로 유명합니다. 제주도는 화산섬으로, 한라산을 중심으로 형성된 화산섬입니다. 이곳에서는 아름다운 해변, 숲, 그리고 다양한 식물을 감상할 수 있습니다.

세 번째로는 **부산**이 있습니다. 부산은 한국의 동쪽에 위치한 도시로, 아름다운 해변과 현대적인 도시 경관으로 유명합니다. 부산에서는 해운대 해변, 부산 타워, 그리고 부산 국제시장 등을 방문할 수 있습니다.

이 세 곳은 한국의 다양한 매력을 경험할 수 있는 대표적인 관광지입니다.


### 2) 모델 파라미터 설정
##### Temperature 효과
* 0.2 (낮은 값): 더 예측 가능하고 일반적인 이야기
* 1.2 (높은 값): 새로운 요소가 등장하며, 더 창의적이고 흥미로운 이야기 생성

##### Presence Penalty 효과
* 0.0 (낮은 값): 기존에 자주 등장하는 단어와 구조 유지
* 1.5 (높은 값): 새로운 표현과 독창적인 아이디어 등장

In [3]:
from langchain_openai import ChatOpenAI

#  보수적인 설정 (일관된, 논리적인 이야기)
llm_before = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=0.2,  # 낮은 온도로 예측 가능한 출력
    presence_penalty=0.0,  # 기존 패턴 유지
    frequency_penalty=0.0,  # 반복 허용
    max_tokens=150,  # 출력 길이 제한
    top_p=1.0  # 확률 상위 100% 내에서 선택 (제한 없음)
)

#  창의적인 설정 (더 독창적이고 예측 불가능한 이야기)
llm_after = ChatOpenAI(
    #model="gpt-3.5-turbo-0125",
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",  # Spring AI와 동일한 모델
    temperature=1.2,  # 높은 온도로 창의적인 답변 유도
    presence_penalty=1.2,  # 새로운 단어와 개념 유도
    frequency_penalty=0.5,  # 반복을 억제하여 더 다양한 표현 생성
    max_tokens=300,  # 더 긴 이야기 허용
    top_p=0.8  # 제한 없이 다양한 단어 선택 가능
)

# 질문 설정: 짧은 판타지 이야기 생성
# question = "마법의 세계에서 용이 인간과 친구가 되는 짧은 이야기를 써 주세요."
question = "마법의 세계에서 벌어지는 예상치 못한 사건을 주제로 독창적인 짧은 이야기를 만들어 주세요."

# 모델 호출
response_before = llm_before.invoke(question)
response_after = llm_after.invoke(question)

# 결과 출력
print(" Before (논리적이고 보수적인 이야기)")
print(response_before.content)

print("\n-------------------------\n")

print(" After (창의적인 이야기, 더 풍부한 표현)")
print(response_after.content)

 Before (논리적이고 보수적인 이야기)
**마법의 세계에서 벌어지는 예상치 못한 사건**

마법의 세계에서 벌어지는 예상치 못한 사건은 마법사들이 사용하는 마법의 도구들이 갑자기 반란을 일으킨 사건이었다. 이 사건은 마법사들이 사용하는 마법의 도구들이 갑자기 반란을 일으켜 마법사들을 공격하기 시작한 사건이었다.

이 사건은 마법사들이 사용하는 마법의 도구들이 갑자기 반란을 일으켜 마법사들을 공격하기 시작한 사건이었다. 마법사들은 자신들이 사용하는 도구들이 반란을 일으킬 것이라고는 생각하지 못했기 때문에 매우 놀랐다.

마법사들은 자신들이 사용하는 도구들이 반란을 일으킬 것이라고는 생각하지 못

-------------------------

 After (창의적인 이야기, 더 풍부한 표현)
어느 날, 마법의 숲 속에서 마법사들이 모여 비밀을 공유하는 지하 모임이 열렸습니다. 이곳은 신비로운 마법사들이 비밀리에 모여 마법의 지식을 공유하는 곳이었죠. 그 날은 달이 차고 풍성한 마법의 밤이었습니다.

이날의 지하 모임에는 유명한 마법사들이 모두 참석했습니다. 엘리노어, 마법의 공주; 마리우스, 마법의 연구자; 그리고 피에르, 마법의 명인 등 많은 마법사들이 있었습니다.

모임이 시작되자마자, 갑자기 지하 공간이 흔들리기 시작했습니다. 처음에는 지진인 줄 알았지만, 그게 아니었습니다. 지하 공간의 벽이 갑자기 무너져 내리고, 그 속에서 거대한 마법의 수정이 나타났습니다.

수정은 지하 공간을 밝히며 빛을 발했습니다. 그리고 마법사들은 그 수정의 중심에서 신비로운 소리를 들었습니다. 그것은 옛날 마법사들이 남긴 비밀 메시지였습니다.

메시지는 마법의 세계를 구할 수 있는 비밀을 담고 있었습니다. 마법사들은 이 메시지를 해독하기 위해 협력하기로 결심했습니다.

엘리노어는 마법의 공주답게 강력한 마법으로 수정을 분석했고, 마리우스는 연구를 통해 메시지의 의미를 밝혀냈습니다. 피에르는 명인답게


#### Model의 종류에 따라 결과 값이 다름
* gpt-4o vs gpt-3.5-turbo-0125

In [4]:
from langchain_openai import ChatOpenAI

# 변경된 설정 (더 창의적인 답변, 새로운 아이디어 유도)
#llm_after = ChatOpenAI(model="gpt-4o", temperature=1.0, presence_penalty=1.5)
#llm_after = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=1.0, presence_penalty=1.5)
llm_after = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=1.0,
    presence_penalty=1.5
)

# 질문 설정
question = "한국에서 가볼 만한 여행지를 추천해 주세요."

# 모델 호출
response_after = llm_after.invoke(question)

# 결과 출력
print("\n-------------------------\n")
print(" After (창의적인 응답, 새로운 아이디어 포함)")
print(response_after.content)


-------------------------

 After (창의적인 응답, 새로운 아이디어 포함)
한국에는 다양한 여행지가 있습니다. 서울의 경복궁, 부산의 해운대, 제주도의 성산 일출봉, 그리고 여수시의 오동도 등이 있습니다. 각 지역마다 고유한 문화와 자연 경관을 가지고 있어 방문객들에게 다양한 경험을 제공합니다. 여행을 계획할 때는 관심사와 계절을 고려하여 선택하는 것이 좋습니다.


##### 2-1) 모델에 직접 파라미터를 전달 (모델 생성 시점)
##### Before / After 파라미터 차이
* temperature: Before(0.7) → 다양성 높은 추천 / After(0.3) → 정확한 일정 제공
* max_tokens: Before(300) → 간략한 응답 / After(800) → 세부 일정 포함
* frequency_penalty: Before(0.5) → 단어 반복 억제 / After(0.2) → 좀 더 자연스러운 응답
* presence_penalty: Before(0.5) → 다양한 장소 추천 / After(0.3) → 새로운 정보 포함

In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

#  모델 파라미터 (Before: 기본적인 추천 / After: 맞춤형 세부 정보 추가)
before_params = {
    "temperature": 0.7,         # 랜덤성을 적절히 유지 (다양한 답변 유도)
    "max_tokens": 300,          # 응답 길이를 적절히 조절
    "frequency_penalty": 0.5,   # 반복 단어 억제
    "presence_penalty": 0.5,    # 새로운 단어 포함 장려
}

after_params = {
    "temperature": 0.3,         # 창의성을 낮추고 정확한 답변 유도
    "max_tokens": 800,          # 더 긴 답변을 생성 (세부 정보 포함)
    "top_p": 0.85,              # 확률 기반 샘플링 (일관성과 다양성 균형)
    "frequency_penalty": 0.2,   # 반복 단어 감소 (자연스러운 답변)
    "presence_penalty": 0.3,    # 새로운 정보 포함 장려
}

#  두 개의 모델 생성 (Before / After)
#before_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **before_params)
before_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **before_params
)
#after_model = ChatOpenAI(model="gpt-3.5-turbo-0125", **after_params)
after_model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    **after_params
)

#  프롬프트 구성 (올바른 ChatMessagePromptTemplate 사용)
system_message = SystemMessagePromptTemplate.from_template(
    "당신은 여행 전문가입니다. 사용자의 요청에 맞는 최적의 여행지를 추천해 주세요."
)

user_message = HumanMessagePromptTemplate.from_template("{user_input}")

chat_prompt = ChatPromptTemplate.from_messages([system_message, user_message])

#  체인 생성 (Before / After)
before_chain = chat_prompt | before_model
after_chain = chat_prompt | after_model

#  질문 설정 (Before / After의 차이점을 비교할 수 있도록 변경)
test_question = "가족과 함께 3박 4일 동안 한국에서 여유롭게 여행할 수 있는 일정을 동선을 고려하여 자세하게 추천해 주세요."

#  Before 모델 실행
before_response = before_chain.invoke({"user_input": test_question})

#  After 모델 실행
after_response = after_chain.invoke({"user_input": test_question})

#  결과 출력
print(" [Before] 모델 결과")
print(before_response.content)
print("\n" + "="*80 + "\n")  # 가독성을 위한 구분선
print(" [After] 모델 결과")
print(after_response.content)

 [Before] 모델 결과
### **추천 여행지: 부산** 

부산은 해운대, 광안리 등 여러 해변과 다양한 문화시설을 제공하는 한국의 대표적인 관광지 중 하나입니다. 특히 가족과 함께 여유로운 여행을 즐기기에 매우 적합한 곳입니다.

### **일정: 3박 4일**

**1일차: 부산 도착 및 해운대 방문**

*   오전: 부산에 도착합니다. 
*   오후: 해운대 해수욕장을 방문하여 바다를 감상하고, 해운대 동백섬으로 이어지는 산책로를 걸으며 여유로운 시간을 보냅니다. 
*   저녁: 해운대 인근의 맛있는 저녁 식사 후, 해운대 야경을 즐깁니다.

**2일차: 광안리 및 부산 대학교**

*   오전: 광안리 해수욕장을 방문하여 서핑을 즐기거나, 광안대교를 감상합니다. 
*   오후: 부산대학교 캠퍼스를 방문해 학생들과의 교류나 학교의 분위기를 느껴봅니다. 
*   저녁: 광안리 인근의 맛있는 저녁 식사 후, 광안리 야경을 즐깁니다.

**3일차: 부산 타워 및 자갈치시장**

*   오전: 부산 타워를 방문하여 부산의 전경을 감상합니다. 
*   오후: 자갈치 시장을 방문하여 신선한 해산물과


 [After] 모델 결과
### **3박 4일 가족 여행 추천 일정**

*   **여행지:** 서울
*   **여행일정:** 3박 4일

### **1일차: 서울 도심 관광**

*   **아침 식사:** 서울역 인근의 맛있는 한식집에서 전통적인 한국식 아침 식사
*   **서울역 출발:** 서울역에서 출발하여 도보로 가까운 거리에 있는 **국립중앙박물관** 방문
*   **오후 식사:** **광화문** 근처 식당에서 점심 식사
*   **광화문 광장:** 광화문 광장에서 잠시 휴식
*   **경복궁:** **경복궁** 방문 (입장료: 무료, 단, 경복궁 내 일부 시설은 유료)
*   **저녁 식사:** **종각** 근처의 식당에서 저녁 식사

### **2일차: 서울의 자연과 문화**

*   **아침 식사:** 호텔에서 조식 뷔페 또는 인근 식당에서 

#### 2-2) 모델에 추가적인 파라미터를 전달
- bind() 메서드를 사용
- max_tokens=50 (기본) / max_tokens=150 (상세한 설명) 
- stop=["."] → 첫 번째 마침표에서 응답을 중단
- temperature=0.8 → 보다 창의적인 답변을 생성함

In [11]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

#  프롬프트 설정 (천문학 질문에 대한 답변을 생성하는 시스템)
prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 대해 명확하고 자세한 답변을 제공할 수 있습니다."),
    ("user", "{user_input}"),
])

#  기본 모델 설정 (기본적인 답변 생성)
#base_model = ChatOpenAI(model="gpt-3.5-turbo-0125", max_tokens=150)  # 150 토큰 제한
base_model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    max_tokens=150
)

#  질문 설정
# 1. MAX_TOKENS 차이를 보여주는 질문 (길이 제한 효과)
max_tokens_question = "인공지능의 발전이 미래 사회에 미칠 영향을 긍정적 측면과 부정적 측면으로 나누어 자세히 설명해 주세요."

# 2. STOP 파라미터 차이를 보여주는 질문 (중단점 효과)
stop_question = "Python 프로그래밍을 배우는 초보자에게 추천하는 학습 단계를 순서대로 설명해 주세요. 각 단계별로 구체적인 방법과 팁을 포함해서 답변해 주세요."

# 3. TEMPERATURE 차이를 보여주는 질문 (창의성 vs 정확성)
temperature_question = "시간 여행이 가능하다면 과거의 어느 시대로 가고 싶은지와 그 이유를 창의적으로 설명해 주세요."

# 4. 복합적 비교를 위한 질문 (모든 파라미터 효과)
complex_question = "화성에 인류가 정착하기 위해 필요한 기술과 준비사항들을 단계별로 설명하고, 각 단계에서 예상되는 도전과제와 해결방안을 제시해 주세요."


question = stop_question

#  Before (기본 max_tokens=150)
messages = prompt.format_messages(user_input=question)
before_answer = base_model.invoke(messages)

#  Before 출력
print("\n [Before] 기본 max_tokens=150 (기본 답변)")
print(before_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

#  After (파라미터 조정 후 비교)
stop_chain = prompt | base_model.bind(max_tokens=150, stop=["."])  # 첫 번째 마침표에서 중단
temp_chain = prompt | base_model.bind(max_tokens=150, temperature=0.8)  # 창의적인 답변 유도

stop_answer = stop_chain.invoke({"user_input": question})
temp_answer = temp_chain.invoke({"user_input": question})

#  After 출력 (stop vs temperature 비교)
print(" [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)")
print(stop_answer.content)

print("\n" + "="*100 + "\n")  # 가독성을 위한 구분선

print(" [After] max_tokens=150, temperature=0.8 (창의적인 답변)")
print(temp_answer.content)


 [Before] 기본 max_tokens=150 (기본 답변)
### **1. Python에 익숙해지기**

**목표:** Python 기초 문법과 변수 선언, 데이터 타입 이해하기

*   **변수 선언:** Python은 변수 선언이 따로 필요 없지만, 변수명을 정할 때는 설명이 쉽고 간결한 이름을 사용하는 것이 좋습니다. 예를 들어 `num_students` 또는 `average_grade`와 같은 변수명을 사용할 수 있습니다.

*   **데이터 타입:** Python의 기본 데이터 타입에는 정수(int), 실수(float), 문자열(str), 리스트(list), 딕셔너리(dict) 등이 있습니다. 초보자는 각 데이터 타입의 특징과 사용법을 이해하는 것이 중요합니다.

*   **주석:** Python에서 코드를 설명하거나 실행하지 않으려는 경우에는


 [After] max_tokens=150, stop=['.'] (첫 번째 마침표에서 응답 중단)
### Python 배워보기

*   **Step 1: Python의 기본 이해** *   **방법:** Python의 공식 홈페이지에서 다운로드 및 설치를 진행하고, 간단한 Python 코드를 실행해 보세요


 [After] max_tokens=150, temperature=0.8 (창의적인 답변)
### **Python 초보자를 위한 단계별 학습 가이드**

1.  **기초 문법 학습** *   **학습 내용**: 변수, 자료형, 제어 구조(조건문, 반복문), 함수 *   **방법**: *   온라인 강의: Codecademy, Coursera, edX 등에서 제공되는 Python 기초 강의 활용 *   교재: 'Python Crash Course' 또는 'Automate the Boring Stuff with Python' 등 *   공식 문서: Python 공식 웹사이트의 튜토리얼 섹션 활용 *   **팁**: 기초 문법을 빠르게 익히기 위해 하루에 30분씩 꾸준히 연습하는 것이 좋습니다.

2